In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ibge.silver

In [0]:
# Import das funções
from pyspark.sql.functions import col, trim, expr
from pyspark.sql.types import IntegerType

In [0]:
#Lendo DataFrame na Silver
anos_estudo_df = spark.table("ibge.bronze.anos_estudo")

In [0]:
# Remover header residual
anos_estudo_linha_df = (
    anos_estudo_df
    .filter(col("NC") != "Nível Territorial (Código)")
)

# Padronização + Safe Cast
anos_estudo_final_df = (
    anos_estudo_linha_df

    # Renomeações
    .withColumnRenamed("NC", "cod_nivel_territorial")
    .withColumnRenamed("NN", "nivel_territorial")
    .withColumnRenamed("MN", "unid_medida")
    .withColumnRenamed("V", "valor")
    .withColumnRenamed("D1C", "cod_municipio")
    .withColumnRenamed("D1N", "municipio")
    .withColumnRenamed("D2N", "variavel")
    .withColumnRenamed("D3N", "ano")
    .withColumnRenamed("D4N", "sexo")
    .withColumnRenamed("D5N", "cor_raca")
    .withColumnRenamed("D6N", "grupo_idade")

    # Trim antes do cast
    .withColumn("valor", trim(col("valor")))

    # Safe Cast para evitar erro com "-", "..."
    .withColumn("valor", expr("try_cast(valor as double)"))

    # Cast das colunas numéricas
    .withColumn("cod_municipio", col("cod_municipio").cast(IntegerType()))
    .withColumn("ano", col("ano").cast(IntegerType()))
    .withColumn("cod_nivel_territorial", col("cod_nivel_territorial").cast(IntegerType()))

    # Trim nas strings
    .withColumn("municipio", trim(col("municipio")))
    .withColumn("variavel", trim(col("variavel")))
    .withColumn("nivel_territorial", trim(col("nivel_territorial")))
    .withColumn("unid_medida", trim(col("unid_medida")))
    .withColumn("sexo", trim(col("sexo")))
    .withColumn("cor_raca", trim(col("cor_raca")))
    .withColumn("grupo_idade", trim(col("grupo_idade")))
)


In [0]:
# Caminho gerenciado por tabela
tabela_silver = "ibge.silver.anos_estudo"

(
    anos_estudo_final_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(tabela_silver)
)